In [1]:

import torch
#assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
     

C:\Users\amb\anaconda3\envs\fastRcnnVisiual\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.chdir(r"C:\Users\amb\Downloads\FasterRcnn\water-body-json\tree-json\data\10-img-test\detecrton\detectron-visiual\detectron2")
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [3]:
pwd

'C:\\Users\\amb\\Downloads\\FasterRcnn\\water-body-json\\tree-json\\data\\10-img-test\\detecrton\\detectron-visiual\\detectron2'

In [4]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = img_anns['imageHeight']
        record["width"] = img_anns['imageWidth']
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # polty for segmentaion
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [5]:

classes = ['water', 'tree']

data_path = 'Dataa/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "my_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("my_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("my_train")

In [6]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from matplotlib.patches import Rectangle
     

In [7]:
cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) 
cfg.DATASETS.TRAIN = ("my_train") 
cfg.DATASETS.TEST = ()
#cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.GAMMA = 0.05 
cfg.SOLVER.STEPS = [500] # milestones where the LR is reduced
#cfg.TEST.EVAL_PERIOD = 200 

cfg.SOLVER.MAX_ITER = 2000 # number of iterations
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.DEVICE = "cpu"

In [8]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) 
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) 

[02/06 20:09:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[02/06 20:09:52 d2.data.build]: Removed 0 images with no usable annotations. 10 images left.
[02/06 20:09:52 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   water    | 5            |    tree    | 40           |
|            |              |            |              |
|   total    | 45           |            |              |
[02/06 20:09:52 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[02/06 20:09:52 d2.data.build]: Using training sampler TrainingSampler
[02/06 20:09:52 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common.NumpySerializedList'>
[02/06 20:09:52 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[02/06 20:09:52 d2.data.common]

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

In [10]:
#import tensorboard
#!pip install tensorboard --user
#!pip install tensorflow
#!pip install tensorboard
import tensorboard

In [11]:
%load_ext tensorboard 
#%reload_ext tensorboard
%tensorboard --logdir output


In [9]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # threshhold for detection
cfg.DATASETS.TEST = ("my_test" ) 
predictor = DefaultPredictor(cfg) 
test_metadata = MetadataCatalog.get("my_test")
test_dataset_dicts = get_data_dicts(data_path+'test', classes)

[02/06 20:10:08 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [13]:
test_dataset_dicts[0]

{'file_name': 'Dataa/test\\lake_001_jpg.rf.e321716b0a5445dd50687e8445aa86f7.jpg',
 'height': 640,
 'width': 640,
 'annotations': [{'bbox': [236.53225806451613,
    212.74193548387098,
    530.8870967741935,
    423.2258064516129],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0}]}

In [14]:
for d in random.sample(test_dataset_dicts, 4):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.8, 
                   #instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #plt.figure(figsize = (20, 10))
    #cv2.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    #cv2.show()
    cv2.imshow("imgTest",v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

C:\Users\amb\anaconda3\envs\fastRcnnVisiual\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [15]:
outputs["instances"]

Instances(num_instances=1, image_height=640, image_width=640, fields=[pred_boxes: Boxes(tensor([[ 43.0809,  55.1838, 517.6102, 542.9277]])), scores: tensor([0.8961]), pred_classes: tensor([0])])

In [17]:
from detectron2.utils.visualizer import ColorMode
import glob
#from google.colab.patches import cv2_imshow
for imageName in glob.glob('w.jpg'):
    
    im = cv2.imread(imageName)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("AnotherImg",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [19]:
outputs

{'instances': Instances(num_instances=1, image_height=640, image_width=640, fields=[pred_boxes: Boxes(tensor([[177.4034, 178.3261, 439.8022, 434.3561]])), scores: tensor([0.9866]), pred_classes: tensor([1])])}

### Real time

In [11]:
import cv2 # import opencv
cap = cv2.VideoCapture(0)#open your laptop camera

#print(cap.isOpened()) #check if the camera is opened
while(cap.isOpened()): #while loop to read all frames
    
    ret, frame = cap.read() #read all frams if ret is TRUE it means that there is a frames to process   
    if ret == True: #if ret is true  
        outputs = predictor(frame)
        counter = []
        water = []
        for key, value in outputs.items():
            tree = value.get('pred_classes')
            for o in tree:
                if o==1:
                    counter.append(1)
                else:
                    if o==0:
                        water.append(1)
        cv2.putText(frame, 'number of trees is='+str(np.sum(counter)), (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1 , (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, 'number of water spot is='+str(np.sum(water)), (40, 80), cv2.FONT_HERSHEY_SIMPLEX, 1 , (255, 0, 0), 2, cv2.LINE_AA)            
        v = Visualizer(frame[:, :, ::-1],metadata=test_metadata, scale=0.8)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        plt.figure(figsize = (20, 10))
        plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
        plt.show()
        m = value.get('pred_boxes')
          #print(tree[0])
        for k in m:
            print(k)
      
            #print(v)
            plt.show()
            e = k.cpu().numpy()
            X , Y , W , H = int(e[0]),int(e[1]),int(e[2]),int(e[3])
            print([X , Y , W , H])
            #cv2.imwrite('image1.jpg',img[:, :, ::-1])
            cropped_image = frame[Y:Y+H , X:X+W]
            #for x in range(0,len(10)):
            plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
            cv2.imwrite('saved images'+str(k)+'.jpg',cropped_image)
        cv2.imshow("frame",v.get_image()[:, :, ::-1])
        
        
        
        #print(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #get the frame width       
        #print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #get the frame height       


        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert frames from BGR2GRAY       
        #cv2.imshow('frame', frame) #show the frames       

        if cv2.waitKey(30)& 0xFF== ord('q'): #if u pressed q close the window and break   
            break          
    

cap.release() #close the camera after u finish the running process
cv2.destroyAllWindows() #close any opened window


C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


tensor([161.4696,  65.2596, 565.9175, 441.2351])
[161, 65, 565, 441]


C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


tensor([ 73.5819,  56.5338, 628.9153, 477.3260])
[73, 56, 628, 477]


C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


...

tensor([  4.7102, 430.4672, 124.0683, 477.5500])
[4, 430, 124, 477]


C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


tensor([  2.2084, 318.2350,  61.5630, 363.9605])
[2, 318, 61, 363]


C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\amb\AppData\Local\Temp\ipykernel_3768\4222625062.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


tensor([104.1292,  60.1778, 623.2480, 470.4778])
[104, 60, 623, 470]
